# Imports and Helper Functions



In [ ]:
%matplotlib inline
from ipywidgets import interactive_output
import ipywidgets as widgets
import numpy as np
from matplotlib import pyplot as plt
from sklearn import linear_model

#Linear Regression



In [ ]:
N = 100
datasets = {}

X = 20 * (np.random.rand(100, 1) - 0.5)  # creates random values in [-10, 10) interval.
X[10] = 8 #OUTLIER

Y = 0.5 * X - 1.2 + np.random.randn(100, 1)
datasets["Linear"] = (X, Y)

Y = 0.1 * (X ** 2) + 0.5 * X - 2 + np.random.randn(100, 1)
datasets["Quadratic"] = (X, Y)

Y = 0.2 * (X ** 3) + 0.6 * (X ** 2) - 0.5 * X - 1 + np.random.randn(100, 1)
datasets["Cubic"] = (X, Y)

Y = 0.5 * X - 1.2 + np.random.randn(100, 1)
Y[10] = -8
datasets["Outlier"] = (X, Y)

In [ ]:
def predict(X, weight, bias):
    #Compute Yhat given X and parameters
    Yhat = weight * X + bias
    return Yhat


def squared_loss(Y, Yhat):
    #Compute the empirical risk given Y and Yhat
    R = np.mean((Y - Yhat) ** 2)
    return (R)


def fit(X, Y):
    #Learn the model parameters
    N = X.shape[0]
    X = np.hstack((X, np.ones((N, 1))))  #Bias absorbtion!
    theta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(Y)
    weight = theta[0]
    bias = theta[1]
    return weight, bias

In [ ]:
def plot_model(weight=0, bias=0, dataset=None, N=0, learn=False, show_residuals=False):
    plt.figure(figsize=(10, 6))
    plt.rcParams.update({'font.size': 12})

    if dataset is not None and N > 0:
        X, Y = datasets[dataset]
        X = X[:N]
        Y = Y[:N]

        if N >= 2 and learn:
            weight, bias = fit(X, Y)

        Yhat = predict(X, weight, bias)
        R = squared_loss(Y, Yhat)
        plt.plot(X, Y, 'ko')

        if show_residuals:
            for n in range(N):
                plt.plot([X[n], X[n]], [Y[n], Yhat[n]], 'k-', alpha=0.5)

    else:
        R = 0

    xs = np.linspace(-10, 10, 200)
    ys = xs * weight + bias

    plt.plot(xs, ys, '-b')
    plt.grid("True")
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    plt.title("weight: %.2f  bias: %.2f  Empirical Risk: %.2f" % (weight, bias, R))
    plt.show()

ww = widgets.FloatSlider(value=0, min=-1, max=1, step=.1, description="Weight", continuous_update=False)
wb = widgets.FloatSlider(value=0, min=-10, max=10, step=.1, description="Bias", continuous_update=False)
wd = widgets.Dropdown(options=["Linear", "Quadratic","Cubic","Outlier"], description="Dataset")
wn = widgets.IntSlider(value=0, min=-1, max=N, step=1, description="N", continuous_update=False)
wl = widgets.Checkbox(value=False, description="Learn")
wr = widgets.Checkbox(value=False, description="Residuals")

out = interactive_output(
    plot_model,
    {"weight": ww, "bias": wb, "dataset": wd, "N": wn, "learn": wl, "show_residuals": wr}
)
out.layout.height = '600px'
box1 = widgets.HBox([ww, wb, wr])
box2 = widgets.HBox([wd, wn, wl])
ui = widgets.VBox([box1, box2])

In [ ]:
display(ui,out)

Output(layout=Layout(height='600px'))

#Linear Classifier

In [ ]:
N = 100
datasets={}

X = np.vstack((2.5*np.random.randn(int(N/2),2)-5, (2.5*np.random.randn(int(N/2),2)+5)))
Y = np.hstack((-1*np.ones((int(N/2))), np.ones((int(N/2)))))

ind = np.random.permutation(range(N))

datasets["Dataset 1"]=(X[ind,:],Y[ind])

X = np.vstack((2.5*np.random.randn(int(N/2),2)-2, (2.5*np.random.randn(int(N/2),2)+1.5)))
Y = np.hstack((-1*np.ones((int(N/2))), np.ones((int(N/2)))))

ind = np.random.permutation(range(N))

datasets["Dataset 2"]=(X[ind,:],Y[ind])

curve1 = np.zeros((int(N/2), 2))
curve1[:,0] = np.arange(-int(N/2),int(N/2),2)/5
curve1[:,1] = (np.arange(-int(N/2),int(N/2),2)**2/150) + np.abs(2.5*np.random.randn(int(N/2)))
curve2 = np.zeros((int(N/2), 2))
curve2[:,0] = np.arange(-int(N/2),int(N/2),2)/5
curve2[:,1] = (np.arange(-int(N/2),int(N/2),2)**2/150) - np.abs(2.5*np.random.randn(int(N/2)))
X = np.vstack((curve1, curve2))
Y = np.hstack((-1*np.ones((int(N/2))), np.ones((int(N/2)))))

ind = np.random.permutation(range(N))

datasets["Dataset 3"]=(X[ind,:],Y[ind])

In [ ]:
def discriminant(X,w0,w1,b):
  Yhat = w0*X[:,0] + w1*X[:,1] +b
  return Yhat

def classification_error(Y,Yhat):
  #Compute the empirical risk given Y and Yhat
  R = np.mean(Y*Yhat<0)
  return(R)

def fit(X,Y):
  clf = linear_model.LogisticRegression(C=1e10)
  clf.fit(X,Y)
  w0=clf.coef_[0,0]
  w1=clf.coef_[0,1]
  b=clf.intercept_
  return(w0,w1,b)

In [ ]:
def plot_model(w0=0, w1=0, b=0, dataset=None, N=0, learn=True):

    plt.figure(figsize=(10,6))
    plt.rcParams.update({'font.size': 12})

    if(dataset is not None and N>0):
      X,Y=datasets[dataset]
      X=X[:N]
      Y=Y[:N]

      ind1=(Y==1)
      indm1=(Y==-1)
      plt.plot(X[ind1,0],X[ind1,1],'bs')
      plt.plot(X[indm1,0],X[indm1,1],'rs')

      if(learn and np.sum(ind1)>0 and np.sum(indm1)>0):
        w0,w1,b=fit(X,Y)

      Yhat = discriminant(X,w0,w1,b)
      R = classification_error(Y,Yhat)

    else:
      R=0

    gx1, gx2 = np.meshgrid(np.arange(-10, 10,1/10.0 ),np.arange(-10, 10,1/10.0))
    gx1l = gx1.flatten()
    gx2l = gx2.flatten()
    gx   = np.vstack((gx1l,gx2l)).T
    gyhat = discriminant(gx,w0,w1,b)
    gyhat = gyhat.reshape(gx1.shape)

    if(not all(gyhat.flatten()==0)):
        contours = plt.contour(gx1, gx2, gyhat, levels=[0], linewidths=2,colors=["k"])
        plt.imshow(gyhat, interpolation='nearest',extent=(-10, 10, -10, 10), aspect='auto', origin='lower', cmap=plt.cm.bwr_r, alpha=0.25)
        plt.clim(-10,10)
        plt.colorbar()

    plt.xlim(-10,10)
    plt.ylim(-10,10)
    plt.title("w0: %.2f w1:%2.f b: %.2f  Error Rate: %.2f"%(w0, w1, b, R))
    plt.xlabel("X0")
    plt.ylabel("X1")
    plt.grid(True)
    plt.legend(["Class 1","Class -1"],loc='upper left')
    plt.show()

ww0=widgets.FloatSlider(value=0.1,min=-1,max=1,step=.1,description="w0",continuous_update=False)
ww1=widgets.FloatSlider(value=0,min=-1,max=1,step=.1,description="w1",continuous_update=False)
wb=widgets.FloatSlider(value=0,min=-10,max=10,step=.1,description="b",continuous_update=False)
wd=widgets.Dropdown(options=["Dataset 1", "Dataset 2", "Dataset 3"],description="Dataset")
wn=widgets.IntSlider(value=0,min=-1,max=N,step=1,description="N",continuous_update=False)
wl=widgets.Checkbox(value=False,description="Learn")

out = interactive_output(plot_model, {"w0":ww0, "w1":ww1,"b":wb,"dataset":wd,"N":wn,"learn":wl});
out.layout.height = '600px'
box1=widgets.HBox([ww0,ww1,wb])
box2=widgets.HBox([wd,wn,wl])
ui=widgets.VBox([box1,box2])


In [ ]:
display(ui,out)

Output(layout=Layout(height='600px'))